## Pull data with yfinance

In [7]:
import yfinance as yf
from datetime import datetime, timedelta
import pandas as pd
from dateutil.relativedelta import relativedelta
import numpy as np

In [2]:
# function to load datasets from yfinance, and create a single table which will be the basis of the data
def load_yf_data(equity, bond):
    from datetime import datetime
    start_date = datetime(year=1950, month=1, day=1)
    end_date = datetime.today()
    interval = "1d"

    # return equity fund data
    ety = yf.download(tickers=equity, interval=interval, start=start_date, end=end_date).reset_index()
    ety.columns = ety.columns.droplevel(1)
    ety = ety[["Date", "Open"]]
    ety.columns = ["date", "ety_open_price"]

    # return bond funds
    bnd = yf.download(tickers=bond, interval=interval, start=start_date, end=end_date).reset_index()
    bnd.columns = bnd.columns.droplevel(1)
    bnd = bnd[["Date", "Open"]]
    bnd.columns = ["date", "bnd_open_price"] 

    # merge the data
    fund_data = ety.merge(bnd, on="date", how="outer")

    return fund_data
    

In [3]:
# call function to load the data
fund_data = load_yf_data("^GSPC", "^FVX")

C:\Users\mosel\AppData\Local\Temp\ipykernel_19732\3201231607.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  ety = yf.download(tickers=equity, interval=interval, start=start_date, end=end_date).reset_index()
[*********************100%***********************]  1 of 1 completed
C:\Users\mosel\AppData\Local\Temp\ipykernel_19732\3201231607.py:15: FutureWarning: YF.download() has changed argument auto_adjust default to True
  bnd = yf.download(tickers=bond, interval=interval, start=start_date, end=end_date).reset_index()
[*********************100%***********************]  1 of 1 completed


In [2]:
# save down the data and reload for working offline
fund_data.to_csv("fund_data.csv", index=False)

NameError: name 'fund_data' is not defined

In [2]:
# read in data
fund_data = pd.read_csv("fund_data.csv")
fund_data["date"] = pd.to_datetime(fund_data["date"])
fund_data.dtypes

date              datetime64[ns]
ety_open_price           float64
bnd_open_price           float64
dtype: object

## Create pension

In [3]:
# create pension class
class Pension():

    def __init__(self, start, retire, contributions):
        self.start = start
        self.retire = retire
        self.cont = contributions

        # create all dates on which contributions will be made
        dates = []
        current_date = self.start

        while current_date <= self.retire:
            dates.append(current_date)
            current_date += relativedelta(months=1)

        self.cont_dates = dates
        

    def load_data(self, data):
        self.funds = data

    def derisk_strategy(self, target_weight, derisk_years):
        self.derisk_years = derisk_years
        self.derisk_target = target_weight
        self.start_derisk_date = self.retire - relativedelta(years=self.derisk_years)
        self.derisk_months = self.derisk_years * 12
        self.derisk_perc_change = round((1 - self.derisk_target) / self.derisk_months, 6)

    def accumulate(self):

        # create all dates on which contributions will be made
        dates = []
        current_date = self.start

        while current_date <= self.retire:
            dates.append(current_date)
            current_date += relativedelta(months=1)
        
        accum = pd.DataFrame(dates, columns=["date"])

        return accum

In [4]:
# test an example
start_date = datetime(year=1990, month=7, day=13)
retire_date = datetime(year=2025, month=7, day=13)
pension = Pension(start_date, retire_date, 750)
pension.derisk_strategy(0.2, 12)
pension.derisk_perc_change

0.005556

In [6]:
fund_data

,date,ety_open_price,bnd_open_price
0,1950-01-03,16.660000,NaN
1,1950-01-04,16.850000,NaN
2,1950-01-05,16.930000,NaN
3,1950-01-06,16.980000,NaN
4,1950-01-09,17.080000,NaN
...,...,...,...
19140,2025-12-18,6778.060059,3.663
19141,2025-12-19,6792.620117,3.686
19142,2025-12-22,6865.209961,3.711
19143,2025-12-23,6872.410156,3.702


In [21]:
from datetime import datetime, timedelta

def get_nearest_next_weekday(date_input):
    if isinstance(date_input, str):
        dt = datetime.strptime(date_input, '%Y-%m-%d')
    else:
        dt = date_input

    day_idx = dt.weekday()

    if day_idx == 5:     # Saturday
        return dt + timedelta(days=2)
    elif day_idx == 6:   # Sunday
        return dt + timedelta(days=1)
    
    return dt

# initialise lists and variables to store results
purchase_dates = []
# equity values
ety_prices = []
ety_units_purchased = []
ls_ety_units_cumsum = []
ety_units_cumsum = 0
ls_ety_value_cumsum = []

# bond values
bnd_prices = []
bnd_units_purchased = []
ls_bnd_units_cumsum = []
bnd_units_cumsum = 0
ls_bnd_value_cumsum = []

# other variables
derisk_month = 0
ls_all_units_cumsum = []


def get_price(date, col, data=fund_data):

    try: 
        price = fund_data[fund_data["date"] == date][col].item()
    except: 
        # iterate until finding the next day when there is a price
        while len(fund_data[fund_data["date"] == date]) == 0:
            date = date + timedelta(days=1)
        price = fund_data[fund_data["date"] == date][col].item()

    return price, m

for m in pension.cont_dates:

    # get prices
    ety_price = get_price(m, "ety_open_price")[0]
    bnd_price = get_price(m, "bnd_open_price")[0]

    # if not derisking, then perform the full allocation to equity
    if m < pension.start_derisk_date:

        # catch dates and equity price
        purchase_dates.append(m)
        ety_prices.append(ety_price)

        # calculate the number of units purchased
        ety_purchased = pension.cont / ety_price
        ety_units_purchased.append(ety_purchased)

        # calculate cumulative number of units purchased
        ety_units_cumsum += ety_purchased
        ls_ety_units_cumsum.append(ety_units_cumsum)

        # caluclate the cumulative value of pension pot 
        ety_value_cumsum = ety_price * ety_units_cumsum
        ls_ety_value_cumsum.append(ety_value_cumsum)

        # include dummy values for bond data
        bnd_prices.append(0)
        bnd_units_purchased.append(0)
        ls_bnd_units_cumsum.append(0)
        ls_bnd_value_cumsum.append(0)
        ls_all_units_cumsum.append(ety_units_cumsum)

    # else the pension is in derisking phase, and should be adjusted
    else: 

        # get target total volume of units owned
        target_units_purchased = pension.cont / ety_price
        target_units_cumsum = ety_units_cumsum + target_units_purchased

        print(f"Target units cumsum: {target_units_cumsum}")
        
        # get target quantity of bonds
        derisk_month += 1
        target_units_bonds = target_units_cumsum * (pension.derisk_perc_change * derisk_month)
        print(f"Target units bonds: {target_units_bonds}")

        # need to less existing bond purchases here

        # calculate how much these bonds will cost
        bnd_value_purchased = bnd_price * target_units_bonds
        print(bnd_value_purchased)
        
        # purchase the remainder in equities
        ety_units_purchased = (pension.cont - bnd_value_purchased) / ety_price
        print(ety_units_purchased)

        print(ety_units_purchased + target_units_bonds + ety_units_cumsum) 
        
        break

Target units cumsum: 258.13208680289875
Target units bonds: 1.4341818742769055
2.0924713422603234
0.4452917342194655
259.5650228544541


In [9]:
# compile into dataframe
pd.DataFrame(data=list(zip(purchase_dates, ety_prices, ety_units_purchased, ls_ety_units_cumsum, ls_ety_value_cumsum, 
                          bnd_prices, bnd_units_purchased, ls_bnd_units_cumsum, ls_bnd_value_cumsum)), 
             columns=["date", "price", "ety_units_purchased", "ety_units_cumsum", "ety_value_cumsum", "bnd_price", "bnd_units_purchased",
                     "bnd_units_cumsum", "bnd_value_cumsum"])

,date,price,ety_units_purchased,ety_units_cumsum,ety_value_cumsum,bnd_price,bnd_units_purchased,bnd_units_cumsum,bnd_value_cumsum
0,1990-07-13,365.450012,2.052264,2.052264,750.000000,0,0,0,0
1,1990-08-13,335.390015,2.236203,4.288467,1438.308941,0,0,0,0
2,1990-09-13,322.510010,2.325509,6.613976,2133.073468,0,0,0,0
3,1990-10-13,300.029999,2.499750,9.113726,2734.391215,0,0,0,0
4,1990-11-13,319.480011,2.347565,11.461291,3661.653297,0,0,0,0
...,...,...,...,...,...,...,...,...,...
271,2013-02-13,1519.430054,0.493606,255.805661,388678.808929,0,0,0,0
272,2013-03-13,1552.479980,0.483098,256.288759,397883.167283,0,0,0,0
273,2013-04-13,1588.839966,0.472043,256.760801,407951.822799,0,0,0,0
274,2013-05-13,1632.099976,0.459531,257.220332,419809.297572,0,0,0,0


In [83]:
fund_data.query("date == '2013-07-15'")

C:\Users\mosel\AppData\Local\Temp\ipykernel_19732\551135305.py:1: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  fund_data.query("date == '2013-07-15'")


,date,ety_open_price,bnd_open_price
16012,2013-07-15,1679.589966,1.459


In [79]:
# work through the calculation

# set values
ety_value_cumsum = 415427.764508
ety_units_cumsum = 257.685549
ety_price = 1612.150024
bnd_price = 1.459
derisk_month = 1

# get target total volume of units owned
target_units_purchased = pension.cont / ety_price
target_units_cumsum = ety_units_cumsum + target_units_purchased

# get target quantity of bonds
target_units_bonds = target_units_cumsum * (derisk_month * pension.derisk_perc_change)
print(target_units_bonds)

# buy the bonds

1.4342856572824059
0.005556


### Solve with Algebra

In [5]:
# set values to test with
curr_e = 415427.764508
curr_b = 0
price_e = 1679.589966
price_b = 1.459

In [8]:
import numpy as np

def calculate_units_to_buy(curr_e, curr_b, price_e, price_b, contribution, target_bond_ratio):
    # Matrix A represents the coefficients of our unknowns (delta_qe, delta_qb)
    # Eq 1: delta_qe * Pe + delta_qb * Pb = C
    # Eq 2: (1-T)*delta_qb - T*delta_qe = T*curr_e - (1-T)*curr_b
    
    A = np.array([
        [price_e, price_b],
        [-target_bond_ratio, (1 - target_bond_ratio)]
    ])
    
    B = np.array([
        contribution,
        target_bond_ratio * curr_e - (1 - target_bond_ratio) * curr_b
    ])
    
    try:
        delta_q = np.linalg.solve(A, B)
        # If any value is negative, we can't reach the target with contribution alone
        if any(q < 0 for q in delta_q):
            return "Boundary Hit: Use Catch-up Logic"
        return delta_q # Returns [delta_qe, delta_qb]
    except np.linalg.LinAlgError:
        return None

In [10]:
pension.derisk_perc_change

0.005556

In [11]:
calculate_units_to_buy(curr_e, curr_b, price_e, price_b, 750, pension.derisk_perc_change)

'Boundary Hit: Use Catch-up Logic'